<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/06_PyTorch_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門（2）


## 線形回帰モデル
* 線形回帰モデルのmini-batch gradient descentをPyTorchで書いてみる。
* PyTorchのDatasetとDataLoaderの使い方も併せて学ぶ。

## 準備

参考資料:
* PyTorch公式のチュートリアル
 * https://pytorch.org/tutorials/index.html


* reproducibilityについては下記リンク先を参照
 * https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import random_split

%config InlineBackend.figure_format='retina'

np.random.seed(123)
torch.manual_seed(123)

### 人工データを作る
 * $y = w_1 x_1 + w_2 x_2 + b + \epsilon$という式にしたがってデータを生成する。
  * $\epsilon$は正規分布に従うとする。

In [ ]:
# データ数
data_size = 1000

# ランダムな二次元ベクトルの集合を訓練データとして設定
X = 10 * torch.rand(data_size, 2) - 5.0

# 係数と切片の正解（これに近い値が求まればよい）
w_true = torch.tensor([[2.0], [-3.0]])
b_true = torch.tensor([10.0])

# 正規乱数を加えた値がターゲット
y = X @ w_true + b_true + torch.normal(0.0, 2.0, (data_size, 1))

## PyTorchのDatasetクラス
* `torch.utils.data.Dataset`を継承して自分用のデータセットのクラスを定義する。
* 以下の２つの関数を必ず書く。
 * データセットの長さを返す関数`__len__`
 * 与えられたインデックスに対応するアイテムを返す関数`__getitem__`

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

### データセットの分割
* ここでは train : valid : test = 8 : 1 : 1 に分割することにする。
 * この分割の比率に、深い意味はない。

In [ ]:
from torch.utils.data import random_split

dataset = MyDataset(X, y)

test_size = len(dataset) // 10
valid_size = test_size
train_size = len(dataset) - valid_size - test_size
train, valid, test = random_split(dataset, [train_size, valid_size, test_size])

In [ ]:
print(f"train size: {len(train)}, validation size: {len(valid)}, test size:{len(test)}")

* 分割した後は、`torch.utils.data.Dataset`ではなく、`torch.utils.data.dataset.Subset`になる。

In [ ]:
type(train)

* `torch.utils.data.dataset.Subset`については、`.dataset`とすれば、元のデータの内容にアクセスできる。
  * こうしてアクセスできるのは、分割する前の、元のデータセットなので、要注意。

In [ ]:
train.dataset.X[:5]

In [ ]:
train[:5][0]

### 訓練データを可視化

In [ ]:
plt.figure(figsize=(9, 4))

X_train = train[:][0]
y_train = train[:][1]

ax1 = plt.subplot(121)
ax1.scatter(X_train[:,0].numpy(), y_train.numpy(), c="b")
plt.xlabel("x1")
plt.ylabel("y", rotation=0)

ax2 = plt.subplot(122)
ax2.scatter(X_train[:,1].numpy(), y_train.numpy(), c="g")
plt.xlabel("x2")
plt.ylabel("y", rotation=0);

## PyTorchのDataLoaderクラス
* 訓練データをシャッフルしてミニバッチをひとつずつ取り出す処理を、PyTorchのDataLoaderを使って実装する。
 * https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
* 評価に使うデータセットはシャッフルしなくてよい。

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズ
batch_size = 10

# 訓練データだけシャッフル
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

* データローダの長さは、ミニバッチの個数。インスタンスの個数ではない。

In [ ]:
len(train_loader)

* 訓練データの最初のミニバッチだけ見てみる。

In [ ]:
print(next(iter(train_loader)))

## モデルの定義と初期化
* 値を推定したいのは、線形モデル$y = w_1 x_1 + w_2 x_2 + b$の係数$w_1,w_2$と切片$b$。
* そこで、係数と切片を微分可能なテンソルとして用意する。

In [ ]:
w = torch.randn((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
print(w)
print(b)

## 損失関数
* 平均二乗誤差を使う。
* PyTorchで用意されている損失関数については、下のリンク先を参照。
 * https://pytorch.org/docs/stable/nn.html#loss-functions

In [ ]:
criterion = torch.nn.MSELoss()

## 最適化アルゴリズム
* 今回は、SGDを使う。
 * 説明のため、あえて`momentum`は使わない。
* PyTorchで用意されているoptimizerについては、下のリンク先を参照。
 * https://pytorch.org/docs/stable/optim.html#algorithms

In [ ]:
optimizer = torch.optim.SGD([w, b], lr=0.001)

## 評価用のヘルパ関数

* 評価する際には、計算グラフを作る必要はない。
* `with torch.no_grad()`で計算グラフを作らないようにする。

In [ ]:
def evaluate(loader, w, b):
  total_loss = 0.0
  total_size = 0
  for input, target in loader:
    with torch.no_grad():
      output = input @ w + b
      loss = criterion(output, target)
      total_loss += loss.item() * len(target)
      total_size += len(target)
  return total_loss, total_size

## 学習
* ループの内側には、以下の４つを書く
1. 損失関数の値を計算することによって、計算グラフを作る
2. backpropagationの実行
3. パラメータの更新
4. gradientをゼロにする



* コメントアウトしたprint関数を実行させると・・・
 * 本当に勾配を使ってwを更新していることが分かる。

In [ ]:
step = 0
for epoch in range(100):

  train_loss = 0.0
  for input, target in train_loader:
    output = input @ w + b
    loss = criterion(output, target)
    train_loss += loss.item() * len(target) # ロギング用の集計

    print(f"\t step {step}: w before update {list(w.detach().numpy().flatten())}")
    loss.backward()
    print(f"\t\t w.grad {list(w.grad.detach().numpy().flatten())}")
    optimizer.step()
    print(f"\t step {step}: w after update {list(w.detach().numpy().flatten())}")
    optimizer.zero_grad()

    step += 1

  # valid lossの計算
  valid_loss, valid_size = evaluate(valid_loader, w, b)

  # print関数でロギング
  print(f'epoch {epoch+1}: ',
        f'train loss {train_loss/train_size:.4f} ',
        f'validation loss {valid_loss/valid_size:.4f} ',
        f'w={list(w.detach().numpy().flatten())}',
        f'b={b.item():.4f}')

In [ ]:
test_loss, test_size = evaluate(test_loader, w, b)
print(f"test loss {test_loss / test_size:8.4f}")

## TensorBoard
* TensorBoardを使ってみる。
 * https://pytorch.org/docs/stable/tensorboard.html
* 上の線形回帰モデルの学習をもう一度そのままおこない、結果を可視化する。

### TensorBoardのnotebook extensionのロード

In [ ]:
%load_ext tensorboard

### TensorBoardを使う準備
* PyTorchのSummaryWriterを使う。
* デフォルトの設定では、「runs」というディレクトリの下にイベント・ファイルが保存される。
* SummaryWriterのlog_dirというパラメータで、イベント・ファイルを保存するディレクトリを指定することもできる。

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

### モデルと損失関数と最適化アルゴリズムの準備
* （上ですでにおこなったことを、改めてもう一度書いているだけです。）

In [ ]:
w = torch.randn((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w, b], lr=0.001)

### SummaryWriterを使って損失関数の値を記録する
* add_scalarメソッドを使っている。

In [ ]:
for epoch in range(30):

  train_loss = 0.0
  for input, target in train_loader:
    output = input @ w + b
    loss = criterion(output, target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  writer.add_scalar('loss/training', train_loss / train_size, epoch)

  valid_loss, valid_size = evaluate(valid_loader, w, b)
  writer.add_scalar('loss/validation', valid_loss / valid_size, epoch)

### SummaryWriterを閉じる

In [ ]:
writer.close()

### 記録した損失関数の値をプロットする

* 絵が出てくるまで、少し時間がかかるかもしれません・・・。

In [ ]:
%tensorboard --logdir runs

## nn.Sequentialクラス
* requires_grad=Trueでテンソルを作ればモデルを用意することはできる。
* しかし、同じことは、torch.nnを使えばもっとすっきり実現できる。
* まず、nn.Sequentialを使う方法を示す。

### nn.Sequentialのインスタンスとしてモデルを作る
* 下記のようにモデルを作った時点でレイヤのパラメータは初期化されている。
* この初期化には上でセットした乱数のシードが使われている。

In [ ]:
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(2, 1),
    )

In [ ]:
type(model)

In [ ]:
print(model)

In [ ]:
# パラメータがどのように初期化されているかを確認してみる
for p in model.parameters():
  print(p.data)

### 損失関数とoptimizer
* optimizerにはモデルのパラメータを渡す。

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### 学習のループ

In [ ]:
writer = SummaryWriter()

In [ ]:
for epoch in range(30):

  train_loss = 0.0
  for input, target in train_loader:
    output = model(input)
    loss = criterion(output, target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  writer.add_scalar('loss/training', train_loss / train_size, epoch)

  # valid loss
  valid_loss = 0
  with torch.no_grad():
    for input, target in valid_loader:
      output = model(input)
      loss = criterion(output, target)
      valid_loss += loss.item() * len(target)

  writer.add_scalar('loss/validation', valid_loss / valid_size, epoch)

### モデルのグラフを表示させる
* SummaryWriterのadd_graphメソッドを使う。

In [ ]:
# 訓練データの最初のインスタンス（どのインスタンスでもよい）でグラフを作る
writer.add_graph(model, next(iter(train_loader))[0])

In [ ]:
writer.close()

* GRAPHSというタブで計算グラフを見ることができる。
 * 計算グラフのノードをダブルクリックしてみよう。

In [ ]:
%tensorboard --logdir runs

## nn.Moduleクラス
* nn.Moduleを継承するクラスを定義する。
* そしてそのクラスのインスタンスとしてモデルを作る。

In [ ]:
class MyLinearModel(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size)

  def forward(self, input):
    return self.fc(input)

* 自分で重みやバイアスを初期化する場合

In [ ]:
class MyLinearModel(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size)
    self.init_weights()

  def init_weights(self):
    self.fc.weight.data.normal_()
    self.fc.bias.data.zero_()

  def forward(self, input):
    return self.fc(input)

In [ ]:
model = MyLinearModel(2, 1)

In [ ]:
# パラメータがどのように初期化されているかを確認してみる
for p in model.parameters():
  print(p.data)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
writer = SummaryWriter()

In [ ]:
for epoch in range(30):

  train_loss = 0
  for input, target in train_loader:
    output = model(input)
    loss = criterion(output, target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  writer.add_scalar('loss/training', train_loss / train_size, epoch)

  # valid loss
  valid_loss = 0
  with torch.no_grad():
    for input, target in valid_loader:
      output = model(input)
      loss = criterion(output, target)
      valid_loss += loss.item() * len(target)

  writer.add_scalar('loss/validation', valid_loss / valid_size, epoch)

In [ ]:
writer.add_graph(model, next(iter(train_loader))[0])

In [ ]:
writer.close()

In [ ]:
%tensorboard --logdir runs

* これは課題ではないが、GPUを使うように書き直してみよう。

# 予告
* 次の回では、下のPyTorchのチュートリアルを、ほぼそのまま使います。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html